In [ ]:
#import thư viện fastai
from fastai.vision.all import *

In [ ]:
#Tạo đường dẫn tới thư mục plant-pathology-2021-fgvc8 trong input
path = Path("../input/plant-pathology-2021-fgvc8")

#In danh sách thư mục con trong thư mục plant-pathology-2021-fgvc8
path.ls()

In [ ]:
#Đọc dữ liệu file  ../input/plant-pathology-2021-fgvc8/train.csv
train_df = pd.read_csv(path/"train.csv")

#In dữ liệu
train_df.head()

In [ ]:
#In kích thước của mảng
train_df.shape

ở đây chúng ta có thể thấy được có 18632 ảnh dùng để training

In [ ]:
#lọc số lượng của mỗi label
train_df["labels"].value_counts()

Ở đây sử dụng [resized dataset](https://www.kaggle.com/ankursingh12/resized-plant2021) để tiết kiệm thời gian.  
nếu không sử dụng thì sẽ mất khá nhiều thời gian để thay đổi kích thước hình ảnh...😅

Bây giờ nếu chúng ta sử dụng ngay ảnh nói trên để train cho model CNN Classify thì sẽ bị hiện tượng Overfit vì dữ liệu nhiều nhưng đa phần giống nhau. Dẫn đến train sẽ có chất lượng tốt nhưng khi test sẽ thấy không được chính xác.
 
 Chúng ta sẽ thực hiện augment dữ liệu để làm phong phú hơn dữ liệu, tăng tính tổng quát cho model 
bằng RandomResizedCrop vì nó sẽ chia tỷ lệ ngẫu nhiên hình ảnh và cắt nó, sau đó thay đổi kích thước nó thành kích thước yêu cầu.

In [ ]:
#ở đây chúng ta sẽ đưa về kích thước ảnh là 128, cắt kích thước ảnh ngẫu nhiên từ 0.75 dến 1.00 và đưa về tỷ lệ 1:1 so với ảnh cũ
item_tfms = [RandomResizedCrop(128, min_scale=0.75, ratio=(1., 1.))]
batch_tfms = [*aug_transforms(size=128, max_warp=0), Normalize.from_stats(*imagenet_stats)]

dls = ImageDataLoaders.from_df(
    df = train_df,
    folder = "../input/resized-plant2021/img_sz_512",
    item_tfms = item_tfms,
    batch_tfms = batch_tfms,
    
    #Dùng để phân chia dữ liệu 10% dữ liệu train và 90% dữ liệu valid
    splitter = RandomSplitter(valid_pct=0.1),
    label_delim = " ",
    bs=256
)

In [ ]:
dls.show_batch()

In [ ]:
#tạo thư mục "/root/.cache/torch/hub/checkpoints"
!mkdir -p /root/.cache/torch/hub/checkpoints

# mô hình phân loại hình ảnh resnet18
!cp ../input/resnet18/resnet18.pth /root/.cache/torch/hub/checkpoints/resnet18-5c106cde.pth
#!cp ../input/resnet18/resnet34.pth /root/.cache/torch/hub/checkpoints/resnet34-333f7ec4.pth
#!cp ../input/resnet50/resnet50.pth /root/.cache/torch/hub/checkpoints/resnet50-19c8e357.pth

In [ ]:
# xây dựng một model 
learn = cnn_learner(
    dls,
    resnet18,
    metrics=[accuracy_multi, F1ScoreMulti()]
).to_fp16()

In [ ]:
# biểu diễn quan hệ giữa learning_rate và loss. Mục đích để hạn chế việc phỏng đoán và tìm ra điểm khởi đầu learning_rate tốt nhất
learn.lr_find()

    Chúng ta có thể thấy được 1 khoảng đốc xuống rồi đi lên, thì khi đấy tỉ lệ mất mát là nhỏ nhất và tỉ lệ học tập ở mức tương đối cao.
    
    Nếu tỷ lệ học tập được đặt quá thấp, quá trình đào tạo sẽ tiến triển rất chậm vì đang cập nhật rất nhỏ các trọng số trong mạng của mình. Tuy nhiên, nếu tỷ lệ học tập được đặt quá cao, nó có thể gây ra hành vi khác biệt không mong muốn trong chức năng mất mát. Vì tỷ lệ học tập tốt nhất có liên quan đến sự sụt giảm mạnh nhất
    
    Vì thế ở đây chúng ta sẽ đặt từ 1e-1

In [ ]:
# tinh chỉnh lại model
learn.fine_tune(
    7,
    1e-1,
    cbs=[
        #lưu mô hình tốt nhất trong quá trình đào tạo và load nó vào cuối
        SaveModelCallback(),
        #dừng lại nếu sau 3 epochs mà k có cải thiện
        EarlyStoppingCallback(patience=3),
    ],
    freeze_epochs=3
)

In [ ]:
# biểu diễn biểu đồ trainning và validation loss
learn.recorder.plot_loss()

In [ ]:
#in ra kết quả dự đoán
learn.show_results()

In [ ]:
# tạo một đối tượng ClassificationInterpretation để kiểm tra các giá trị 
interp = ClassificationInterpretation.from_learner(learn)

In [ ]:
# in ra 9 loss cao nhất
interp.plot_top_losses(9)

In [ ]:
# biểu diễn xem có bao nhiêu điểm dữ liệu thực sự thuộc vào một class, và được dự đoán là rơi vào một class
interp.plot_confusion_matrix()

In [ ]:
# đọc file sample_submission.csv,là dạng file chúng ta cần tạo để submit
submission_df = pd.read_csv(path/"sample_submission.csv")
submission_df.head()

In [ ]:
# đổi path cho các item trong submission_df["image"]
test_image_path_series = submission_df["image"].apply(lambda x: f"../input/plant-pathology-2021-fgvc8/test_images/{x}")
test_image_path_series.head()

In [ ]:
# đưa kết quả dự đoán (predict) từ dataset test_dl
test_dl = learn.dls.test_dl(test_image_path_series)
preds, _ = learn.tta(dl=test_dl)

In [ ]:
#in ra predictions
preds

In [ ]:
# các label trong Dataloaders của model 
vocab = learn.dls.vocab
vocab

In [ ]:
# hàm quyết định từ kết quả dự đoán được sẽ xếp nó là label nào, sau đó đưa vào mảng labels
threshold = 0.5

def pred_to_labels(pred):
    labels = []
    for i, probability in enumerate(pred):
        if probability > threshold:
            labels.append(vocab[i])
            
    return " ".join(labels)

In [ ]:
# tạo label_list chứa tất cả label mà model dự đoán được 
labels_list = [pred_to_labels(pred) for pred in preds]
labels_list

In [ ]:
# chuyển labels_list vào thành cột labels trong submission dataframe 
submission_df["labels"] = labels_list
submission_df.head()

In [ ]:
# chuyển submission_df sang dạng .csv để submit
submission_df.to_csv("submission.csv", index=False)